## Install h2o
- Java https://www.oracle.com/java/technologies/javase-jdk13-downloads.html (Install v13 or below)
- http://docs.h2o.ai/h2o/latest-stable/h2o-docs/downloading.html#install-in-python
 - (Run in Terminal) pip install -f http://h2o-release.s3.amazonaws.com/h2o/latest_stable_Py.html h2o

# Import Libraries

In [22]:
#Basic Packages
import pandas as pd
import numpy as numpy

#H2O
import h2o
from h2o.automl import H2OAutoML, get_leaderboard

#Evaluation Packages
from sklearn import metrics
from sklearn.metrics import roc_auc_score

In [2]:
#Initialize H2O
h2o.init()

Checking whether there is an H2O instance running at http://localhost:54321 . connected.


H2O cluster uptime:,15 mins 36 secs
H2O cluster timezone:,Asia/Kolkata
H2O data parsing timezone:,UTC
H2O cluster version:,3.28.1.2
H2O cluster version age:,11 days
H2O cluster name:,H2O_from_python_DELL_83xmvp
H2O cluster total nodes:,1
H2O cluster free memory:,3.964 Gb
H2O cluster total cores:,8
H2O cluster allowed cores:,8
H2O cluster status:,"locked, healthy"


# Import train and test Datasets

In [3]:
train = pd.read_csv("train.csv")
test = pd.read_csv("test.csv")

## Check for Missing Values

In [4]:
train.isnull().sum()

PassengerId      0
Survived         0
Pclass           0
Name             0
Sex              0
Age            177
SibSp            0
Parch            0
Ticket           0
Fare             0
Cabin          687
Embarked         2
dtype: int64

In [5]:
test.isnull().sum()

PassengerId      0
Pclass           0
Name             0
Sex              0
Age             86
SibSp            0
Parch            0
Ticket           0
Fare             1
Cabin          327
Embarked         0
dtype: int64

# Treat Missing Values

In [6]:
all = pd.concat([train, test], sort = False)
all.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 1309 entries, 0 to 417
Data columns (total 12 columns):
PassengerId    1309 non-null int64
Survived       891 non-null float64
Pclass         1309 non-null int64
Name           1309 non-null object
Sex            1309 non-null object
Age            1046 non-null float64
SibSp          1309 non-null int64
Parch          1309 non-null int64
Ticket         1309 non-null object
Fare           1308 non-null float64
Cabin          295 non-null object
Embarked       1307 non-null object
dtypes: float64(3), int64(4), object(5)
memory usage: 132.9+ KB


In [7]:
#Fill Missing numbers with median for Age and Fare
all['Age'] = all['Age'].fillna(value=all['Age'].median())
all['Fare'] = all['Fare'].fillna(value=all['Fare'].median())

#Treat Embarked
all['Embarked'] = all['Embarked'].fillna(value=all['Embarked'].mode()[0])

#Bin Age
all.loc[ all['Age'] <= 16, 'Age'] = 0
all.loc[(all['Age'] > 16) & (all['Age'] <= 32), 'Age'] = 1
all.loc[(all['Age'] > 32) & (all['Age'] <= 48), 'Age'] = 2
all.loc[(all['Age'] > 48) & (all['Age'] <= 64), 'Age'] = 3
all.loc[ all['Age'] > 64, 'Age'] = 4 

#Cabin
all['Cabin'] = all['Cabin'].fillna('Missing')
all['Cabin'] = all['Cabin'].str[0]

#Family Size & Alone 
all['Family_Size'] = all['SibSp'] + all['Parch'] + 1
all['IsAlone'] = 0
all.loc[all['Family_Size']==1, 'IsAlone'] = 1

In [8]:
all.isnull().sum()

PassengerId      0
Survived       418
Pclass           0
Name             0
Sex              0
Age              0
SibSp            0
Parch            0
Ticket           0
Fare             0
Cabin            0
Embarked         0
Family_Size      0
IsAlone          0
dtype: int64

## Extra Features: Title

In [9]:
#Extract Title from Name
all['Title'] = all['Name'].str.extract(' ([A-Za-z]+)\.', expand=False)

In [10]:
all['Title'].value_counts()

Mr          757
Miss        260
Mrs         197
Master       61
Dr            8
Rev           8
Col           4
Ms            2
Major         2
Mlle          2
Mme           1
Jonkheer      1
Dona          1
Capt          1
Lady          1
Sir           1
Countess      1
Don           1
Name: Title, dtype: int64

In [11]:
#We will combine a few categories, since few of them are unique 
all['Title'] = all['Title'].replace(['Capt', 'Dr', 'Major', 'Rev'], 'Officer')
all['Title'] = all['Title'].replace(['Lady', 'Countess', 'Don', 'Sir', 'Jonkheer', 'Dona'], 'Royal')
all['Title'] = all['Title'].replace(['Mlle', 'Ms'], 'Miss')
all['Title'] = all['Title'].replace(['Mme'], 'Mrs')
all['Title'].value_counts()

Mr         757
Miss       264
Mrs        198
Master      61
Officer     19
Royal        6
Col          4
Name: Title, dtype: int64

In [12]:
#Drop unwanted variables
all = all.drop(['Name', 'Ticket'], axis = 1)
all.head()

,PassengerId,Survived,Pclass,Sex,Age,SibSp,Parch,Fare,Cabin,Embarked,Family_Size,IsAlone,Title
0,1,0.0,3,male,1.0,1,0,7.2500,M,S,2,0,Mr
1,2,1.0,1,female,2.0,1,0,71.2833,C,C,2,0,Mrs
2,3,1.0,3,female,1.0,0,0,7.9250,M,S,1,1,Miss
3,4,1.0,1,female,2.0,1,0,53.1000,C,S,2,0,Mrs
4,5,0.0,3,male,2.0,0,0,8.0500,M,S,1,1,Mr


## Create Dummy Values
We will drop one of them using drop_first = True

In [13]:
all_dummies = pd.get_dummies(all, drop_first = True)
all_dummies.head()

,PassengerId,Survived,Pclass,Age,SibSp,Parch,Fare,Family_Size,IsAlone,Sex_male,...,Cabin_M,Cabin_T,Embarked_Q,Embarked_S,Title_Master,Title_Miss,Title_Mr,Title_Mrs,Title_Officer,Title_Royal
0,1,0.0,3,1.0,1,0,7.2500,2,0,1,...,1,0,0,1,0,0,1,0,0,0
1,2,1.0,1,2.0,1,0,71.2833,2,0,0,...,0,0,0,0,0,0,0,1,0,0
2,3,1.0,3,1.0,0,0,7.9250,1,1,0,...,1,0,0,1,0,1,0,0,0,0
3,4,1.0,1,2.0,1,0,53.1000,2,0,0,...,0,0,0,1,0,0,0,1,0,0
4,5,0.0,3,2.0,0,0,8.0500,1,1,1,...,1,0,0,1,0,0,1,0,0,0


## Covert Pandas Dataframe to H2O Frame

In [14]:
all_train = h2o.H2OFrame(all_dummies[all_dummies['Survived'].notna()])
all_test = h2o.H2OFrame(all_dummies[all_dummies['Survived'].isna()])

Parse progress: |█████████████████████████████████████████████████████████| 100%
Parse progress: |█████████████████████████████████████████████████████████| 100%


# Train/Test Split

In [15]:
#Get columns names for Building H2O Models
y = 'Survived'
X = [f for f in all_train.columns if f not in ['Survived','PassengerId']]

### Diving the dataset into Train, Validation and Test
- **Train:** will be used to build model
- **Validation** is used to help improve the evaluation metric (We will not use this in this kernel)
- **Test** is used to help us evaluate the model we built

In [16]:
train_df, valid_df, test_df = all_train.split_frame(ratios=[0.7, 0.15], seed=2018)

In [19]:
#Covert dtype to factor as per H2O implementation
train_df[y] = train_df[y].asfactor()
valid_df[y] = valid_df[y].asfactor()
test_df[y] = test_df[y].asfactor()

# Build Model

In [20]:
#Check X Variables
X

['Pclass',
 'Age',
 'SibSp',
 'Parch',
 'Fare',
 'Family_Size',
 'IsAlone',
 'Sex_male',
 'Cabin_B',
 'Cabin_C',
 'Cabin_D',
 'Cabin_E',
 'Cabin_F',
 'Cabin_G',
 'Cabin_M',
 'Cabin_T',
 'Embarked_Q',
 'Embarked_S',
 'Title_Master',
 'Title_Miss',
 'Title_Mr',
 'Title_Mrs',
 'Title_Officer',
 'Title_Royal']

In [23]:
# initialize the H2O GBM 
aml = aml = H2OAutoML(max_models=20, seed=1)

# train with the initialized model
aml.train(x=X, y=y, training_frame=train_df)

AutoML progress: |█
16:10:56.914: AutoML: XGBoost is not available; skipping it.

██████████████████████████████████████████████████████ (cancelled)  99%


KeyboardInterrupt: 

In [ ]:
# AutoML Leaderboard
lb = aml.leaderboard
lb

In [ ]:
#Predict on Test Frame to evaluate how well our model performed
#as_data_frame() converts the data to Pandas DataFrame
test_pred_val = gbm.predict(test_df[predictors])[0].as_data_frame()
test_pred_val

# Check Accuracy

In [ ]:
#Test Accuracy
test_true_val = (test_df[target]).as_data_frame()
prediction_auc = roc_auc_score(test_pred_val, test_true_val)
print(f'Test : {prediction_auc:.3f}')

In [ ]:
#Predict Train
train_pred_val = gbm.predict(train_df[predictors])[0].as_data_frame()
train_pred_val

#Check Train Accuracy 
train_true_val = (train_df[target]).as_data_frame()
prediction_auc = roc_auc_score(train_pred_val, train_true_val)
print(f'Train : {prediction_auc:.3f}')

# Final Predictions for Competition

In [ ]:
#Get X Variables from Competition Test Dataset
TestForPred = all_test.drop(['PassengerId', 'Survived'], axis = 1)

In [ ]:
#Predict
fin_pred = gbm.predict(TestForPred[predictors])[0].as_data_frame()

In [34]:
#Get Competition Test Ids
PassengerId = all_test['PassengerId'].as_data_frame()

In [35]:
#Make Submission File
h2o_Sub = pd.DataFrame({'PassengerId': PassengerId['PassengerId'].tolist(), 'Survived':fin_pred['predict'].tolist() })
h2o_Sub.head()

,PassengerId,Survived
0,892,0
1,893,1
2,894,0
3,895,0
4,896,1


In [36]:
#Export Submission File
h2o_Sub.to_csv("1_h2o_Submission.csv", index = False)